<a href="https://colab.research.google.com/github/elbernaderen/final-project/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instalación de python, librerias con mapas y flask en el servidor

In [2]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
# Install flask-ngrok to run flask on colab
!pip install flask-ngrok
# Installjavascript on colab
!npm install -g --unsafe-perm ijavascript zeromq
!ijsinstall --install=global
! pip install Flask-Session
! pip install Flask

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (1,938 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

# Base de datos

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS claims(id_claim INTEGER,
                      name TEXT,
                      address TEXT, 
                      latitude REAL, 
                      longitude REAL, 
                      date TEXT,
                      reason TEXT,
                      PRIMARY KEY(id_claim));''')

conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS users (id INTEGER,
                              username TEXT,
                              hash TEXT, 
                              PRIMARY KEY(id));''')
conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


# aplicación

https://stackoverflow.com/questions/52638926/geo-coding-for-intersection-street-using-python from here i have taken the idea for searching latitude and longitude from googlemaps

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/final')
import urllib.parse

from flask_ngrok import run_with_ngrok
from flask_session import Session
from datetime import datetime
from tempfile import mkdtemp
from flask import Flask, flash, redirect, render_template, request, session, make_response
from geopandas.tools import geocode
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import requests
import sqlite3
import re
from werkzeug.exceptions import default_exceptions, HTTPException, InternalServerError
from werkzeug.security import check_password_hash, generate_password_hash
from helpers import login_required
from pytz import timezone
import random
import json

app = Flask(__name__,template_folder='/content/drive/MyDrive/final/template')
run_with_ngrok(app)
app.config["TEMPLATES_AUTO_RELOAD"] = True

# Ensure responses aren't cached
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    response.headers["Expires"] = 0
    response.headers["Pragma"] = "no-cache"
    return response

app.config["SESSION_FILE_DIR"] = mkdtemp()
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

lat = -31.4186703
long = -64.1903681


@app.route("/", methods=["GET", "POST"])
def index():
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        m_1 = folium.Map(location = (lat,long), zoom_start = 12)
        m_1.save('/content/drive/MyDrive/final/template/map.html')
        if request.method == "POST":
          name = request.form.get("name")
          time = datetime.now(timezone("America/Argentina/Cordoba"))
          direction = request.form.get("direction")
          reason =  request.form.get("reason")
          gps1 = loc(direction)
          gps1[0] = float(gps1[0])
          gps1[1] = float(gps1[1])
          conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], time, reason])
          conn.row_factory = sqlite3.Row
          cur = conn.cursor()
          cur.execute(''' SELECT id_claim FROM claims WHERE name = ? AND date = ? ;''', [name, time])
          r = cur.fetchone()
          conn.commit()
          return render_template("claimed.html",id_claim = r["id_claim"], name = name)
        else:
          return render_template("index.html")


@app.route("/claims", methods=["GET", "POST"])
@login_required
def claims():
    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    m_1 = folium.Map(location = (lat,long), zoom_start = 12)
    cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
    
    for row in cursor:
          if row[2] == "Sewer overflow":
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
          else:
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)        
    conn.close()
    m_1.save('/content/drive/MyDrive/final/template/map.html')
    return render_template('claims.html')





@app.route("/register", methods=["GET", "POST"])
def register():
    """Register user"""
    if request.method == "POST":
        password = request.form.get("password")
        phash = generate_password_hash(password)
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        conn.execute("INSERT INTO users (username, hash) VALUES(?, ?)", [request.form.get("name"), phash])
        rows = conn.execute("""SELECT id FROM users WHERE username = ?""", [request.form.get("name")])
        row = rows.fetchone()
        session["user_id"] = row[0]
        conn.commit()
        # Redirect user to home page
        return redirect("/claims")
    else:
        return render_template("register.html")

@app.route("/login", methods=["GET", "POST"])
def login():
    """Log user in"""

    # Forget any user_id
    session.clear()

    # User reached route via POST (as by submitting a form via POST)
    if request.method == "POST":
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        # Query database for username
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute("""SELECT * FROM users WHERE username = ?""", [request.form.get("name")])
        r = cur.fetchone()
        conn.commit()      
        # Ensure username exists and password is correct
        if r["username"] == request.form.get("name") and check_password_hash(r["hash"], request.form.get("password")):
          # Remember which user has logged in
          session["user_id"] = r["id"]
          # Redirect user to home page
          return redirect("/claims")
        else:
          return render_template("login.html")
        

    # User reached route via GET (as by clicking a link or via redirect)
    else:
        return render_template("login.html")



@app.route("/logout")
def logout():
    """Log user out"""

    # Forget any user_id
    session.clear()

    # Redirect user to login form
    return redirect("/")

@app.route('/map')
def map():
    return render_template('map.html')


@app.route("/ajax_index", methods=["GET", "POST"])
def ajax_index():
    if request.method == "POST":
      conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
      print("1")
      json_data = json.loads(request.data)
      print(json_data)
      gps1 = loc(json_data["direction"])
      gps1[0] = float(gps1[0])
      gps1[1] = float(gps1[1])
       #   time = datetime.now(timezone("America/Argentina/Cordoba"))
        #  direction = request.form.get("direction")
        #  reason =  request.form.get("reason")
       #   conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], time, reason])
       #   conn.row_factory = sqlite3.Row
      #    cur = conn.cursor()
      #    cur.execute(''' SELECT id_claim FROM claims WHERE name = ? AND date = ? ;''', [name, time])
      #    r = cur.fetchone()
      #    conn.commit()
      #    return render_template("claimed.html",id_claim = r["id_claim"], name = name)
      #  else:


      #m_1 = folium.Map(location = (lat,long), zoom_start = 12)
      #cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
      m_1 = folium.Map(location = (gps1[0],gps1[1]), zoom_start = 120)
      folium.Marker(location=[gps1[0],gps1[1]]).add_to(m_1)
      

      m_1.save('/content/drive/MyDrive/final/template/map.html')
      print("ok")
      return render_template("get_data.html")    

@app.route("/ajax", methods=["GET", "POST"])
def ajax():
    if request.method == "POST":
      conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
      print("1")
      reason_s = request.form.get("data")
      print(reason_s)
      m_1 = folium.Map(location = (lat,long), zoom_start = 12)
      cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
      for row in cursor:
        if reason_s == row[2]:
          folium.Marker(location=[row[0],row[1]],popup = row[2]).add_to(m_1)
        if reason_s == "all":
          if row[2] == "Sewer overflow":
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
          else:
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)
      conn.close()
      m_1.save('/content/drive/MyDrive/final/template/map.html')
      print("ok")
      return render_template("get_data.html")

def loc(direction):
    try:

      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina", cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
      print(res.url)
      print(res.headers)
      return re.findall(r'll=(.*?)" item', res.text)[0].split(',')
    except IndexError:
      
      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina", cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
      print(res.url)
      print(res.headers)
      rest = re.findall(r'null,null,-(.*?)]', res.text)[0].split(',')
      rest[0] = float(rest[0]) * -1  
      return rest

app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://227d442a2730.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Jul/2021 23:06:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:06:54] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:06:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Jul/2021 23:06:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:07:01] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:07:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:07:24] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:02] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:39] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:08:58] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:09:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:09:40] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:10:13] "GET / HTTP/1.1" 200 -
127.0.0.

1
{'name': 'bernardo', 'direction': 'colon 520', 'reason': 'Sewer overflow'}
https://www.google.com/maps/place/colon%20520,+Ciudad+de+C%C3%B3rdoba,+Argentina
{'Date': 'Wed, 21 Jul 2021 23:19:22 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=UTF-8', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2021-07-21-23; expires=Fri, 20-Aug-2021 23:19:22 GMT; path=/; domain=.google.com; Secure, NID=219=xZLYxWDgQLvu-c2v43ZicI7RquYUfk8jd0h6F5J7ffj6jR0HWM4bV809Q959N2R6mObjgL0wPZ1g_cJ7WzqlHEiPiQZiKvGvJZHu3ByQOsB8p71dLoPphuMK2m5mcmejVV0kdI8r5IXwICEes_7eseyIycGjjqeSyOzePwvCJ_Q; expires=Thu, 20-Jan-2022 23:19:22 GMT; path=/; domain=.google.com; HttpOnly', 'Transfer-Encoding': 'chunked'}
ok


...

127.0.0.1 - - [21/Jul/2021 23:31:56] "GET /map HTTP/1.1" 200 -


https://www.google.com/maps/place/Av.%20colon%20y%20Av.%20General%20Paz,+Ciudad+de+C%C3%B3rdoba,+Argentina
{'Date': 'Wed, 21 Jul 2021 23:32:05 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=UTF-8', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2021-07-21-23; expires=Fri, 20-Aug-2021 23:32:05 GMT; path=/; domain=.google.com; Secure, NID=219=v9OkIL7TaE_bPMcYNWhyYr7_HqGpfu_zpg3SsYVI9c5N4vt9RftjXXpfjoszmNNUhzWQJAkRp8rNp8njFtZoXLuNwB-nb1CdnAwErAbl9PI2eMeZNORpk_23avXkZbhgoZkBGMe9YW2THeEYqXfaflczkeWKLbLyQrs0d5QzZUg; expires=Thu, 20-Jan-2022 23:32:04 GMT; path=/; domain=.google.com; HttpOnly', 'Transfer-Encoding': 'chunked'}


127.0.0.1 - - [21/Jul/2021 23:32:05] "POST / HTTP/1.1" 200 -


https://www.google.com/maps/place/Av.%20colon%20y%20Av.%20General%20Paz,+Ciudad+de+C%C3%B3rdoba,+Argentina
{'Date': 'Wed, 21 Jul 2021 23:32:05 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=UTF-8', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2021-07-21-23; expires=Fri, 20-Aug-2021 23:32:05 GMT; path=/; domain=.google.com; Secure, NID=219=LryJG_Bk3RVr1x_w8UktvgG4LMarlyJu8NuY9TrkKV4vWZrZTxFYC3udaMSLXZV1Qab10aSenPhYaYT05w110rbgFf4UiPDcHIV3NZldm4mYMqEmoY-oCiGUkCwZvfzNq1XNutb_D7CifGPKkWfamOXHmZbLAAV8i1Bp9ovTStk; expires=Thu, 20-Jan-2022 23:32:05 GMT; path=/; domain=.google.com; HttpOnly', 'Transfer-Encoding': 'chunked'}


127.0.0.1 - - [21/Jul/2021 23:32:10] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [21/Jul/2021 23:32:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2021 23:32:11] "GET /map HTTP/1.1" 200 -
